Classes de modelo que serão usadas para solicitação da portabilidade

In [ ]:
from pydantic import BaseModel

class Enterprise(BaseModel):
    name: str
    cnpj: str
    address: str

class AccountData(BaseModel):
    user_name: str
    cpf: str
    agency: str
    account: str
    dac: str
    bank_code: str
    enterprise: Enterprise

Funções que serão utilizadas pelas Tools do agente para executar a chamada para o OpenFinance e a solicitação da portabilidade

In [ ]:
import requests
from langchain_core.tools import ToolException

def _get_data_from_openfinance(*args):
    try:
        response = requests.get("http://127.0.0.1:8080/account_data")
        return response.json()
    except:
        return {'message': 'Não foi possivel consultar os dados do cliente no OpenFinance'}

def _request_portability_without_data_imputed(s: str):
    try:
        data = _get_data_from_openfinance()
        payload = {
	        "user_name": data['data']['user_name'],
	        "cpf": data['data']['cpf'],
	        "agency": data['data']['agency'],
	        "account": data['data']['account'],
	        "dac": data['data']['dac'],
	        "bank_code": data['data']['bank_code'],
	        "enterprise": {
	        	"name": data['data']['enterprise']['name'],
	        	"cnpj": data['data']['enterprise']['cnpj'],
	        	"address": data['data']['enterprise']['address']
	        }
        }
        response = requests.post("http://127.0.0.1:8081/effect_portability", json=payload)
        if response.status_code == 200:
            return 'Portabilidade solicitada com sucesso!'
        return 'Não foi possivel solicitar a portabilidade de salário.'
    except:
        return 'Não foi possivel solicitar a portabilidade de salário.'
    
def _request_portability_with_imputed_data(**kwargs):
    try:
        payload = {
	        "user_name": kwargs['user_name'],
	        "cpf": kwargs['cpf'],
	        "agency": kwargs['agency'],
	        "account": kwargs['account'],
	        "dac": kwargs['dac'],
	        "bank_code": kwargs['bank_code'],
	        "enterprise": {
	        	"name": kwargs['enterprise']['name'],
	        	"cnpj": kwargs['enterprise']['cnpj'],
	        	"address": kwargs['enterprise']['address']
	        }
        }
        response = requests.post("http://127.0.0.1:8081/effect_portability", json=payload)
        if response.status_code == 200:
            return 'Portabilidade solicitada com sucesso!'
        return 'Não foi possivel solicitar a portabilidade de salário.'
    except:
        return 'Não foi possivel solicitar a portabilidade de salário.'
    
def _handle_error(error: ToolException) -> str:
    if error.args[0].startswith("Too many arguments to single-input tool"):
        return "Formate em uma ÚNICA STRING JSON. NÃO USE ENTRADA DE MÚLTIPLOS"
    return (
    "The following errors occurred during tool execution:"
    + error.args[0]
    + "Please try another tool.")

Tools que serão usadas pelo Agent para executar a solicitação de portabilidade

In [ ]:
from langchain.agents import Tool
from langchain.tools import StructuredTool

tools = [
    Tool.from_function(
        func=_get_data_from_openfinance,
        name='get_data_from_openfinance',
        description='Utilize essa ferramenta para obter os dados do usuário.',
        handle_tool_error=_handle_error,
    ),
    Tool.from_function(
        func=_request_portability_without_data_imputed,
        name='portability_without_data_imputed',
        description='Utilize esta ferramenta para tentar solicitar a portabilidade salarial quando NÃO possuir os dados do usuário.',
        handle_tool_error=_handle_error,
    ),
    StructuredTool.from_function(
        func=_request_portability_with_imputed_data,
        name='portability_with_data_imputed',
        description='Utilize esta ferramenta para tentar solicitar a portabilidade salarial quando possuir os dados do usuário.',
        args_schema=AccountData,
        handle_tool_error=_handle_error,
    )
]

Inicializa LLM, Memory e Prompt

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# gpt-3.5-turbo
# gpt-4
llm = ChatOpenAI(model='gpt-4')

prompt_template = open('agent_portability_prompt.txt', 'r', encoding='utf-8').read()
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', prompt_template),
        MessagesPlaceholder(variable_name='history'),
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

Criação do Agent que será responsável por tomar as decisões e executar a portabilidade de salário.

In [ ]:
from langchain.agents import AgentType,initialize_agent

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

Cria a Chain com o Prompt, Memory e Agent

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda, RunnablePassthrough

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | agent
)

Função responsável por se comunicar com a IA e salvar o histórico de mensagens

In [ ]:
def _call_ia(input: str):
    response = chain.invoke({'input': input})
    memory.save_context({'input': input}, {'output': response['output']})
    print(response['output'])

In [ ]:
_call_ia("Olá")